# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/data.frame_idx')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float and then to int, before using it below
sales['floors'] = sales['floors'].astype(float).astype(int)
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650.0,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242.0,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000.0,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000.0,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080.0,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930.0,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819.0,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711.0,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470.0,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560.0,2,0


If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X)

[[ 3.  5.  8.]
 [ 4. 12. 15.]]


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [8]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features = feature_matrix / norms
    
    return normalized_features, norms

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

[[0.6 0.6 0.6]
 [0.8 0.8 0.8]]
[ 5. 10. 15.]


# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [13]:
weights = np.array([1., 4., 1.])

Use `predict_output()` to make predictions on this data.

In [14]:
prediction = predict_output(simple_feature_matrix, weights)

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [15]:
for i in range(len(simple_feature_matrix[0])):
    ro = (simple_feature_matrix[:, i] * (output - prediction + (weights[i]*simple_feature_matrix[:, i]))).sum()
    print('for variable {} we have ro = {}'.format(i, ro))

for variable 0 we have ro = 79400300.03492916
for variable 1 we have ro = 87939470.77299108
for variable 2 we have ro = 80966698.67596565


***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

In [16]:
boundery = '{:e}'.format((80966698.67596565+1)*2)
print(boundery)
#numbers greater than:

1.619334e+08


***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

In [17]:
boundery2 = '{:e}'.format((87939470.77299108+1)*2)
print(boundery2)

1.758789e+08


So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [18]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = (feature_matrix[:, i] * (output - prediction + (weights[i]*feature_matrix[:, i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [19]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                    np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910251


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [20]:
import pdb

def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    not_converged = True
    weights = initial_weights
    
    while not_converged:
        #pdb.set_trace()
        difs = []
        for i in range(len(weights)):
            old_weight = weights[i]

            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, 
                                                           l1_penalty)
            
            change = abs(weights[i] - old_weight)
            if change > tolerance:
                difs.append(1)
            else:
                difs.append(0)
        
        if sum(difs) >=1:
            not_converged = True
        else:
            not_converged = False
    
    return weights

Using the following parameters, learn the weights on the sales dataset. 

In [21]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [22]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [23]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)

In [24]:
weights

array([21624998.36636292, 63157246.78545421,        0.        ])

In [25]:
# RSS
errs_2 = (predict_output(normalized_simple_feature_matrix, weights) - output)**2
RSS_2 = errs_2.sum()
print('RSS: {}'.format(RSS_2))

RSS: 1630492481484487.8


# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [26]:
train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [27]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [28]:
(train_matrix, train_output) = get_numpy_data(train_data, all_features, my_output)
(normalized_train, train_norms) = normalize_features(train_matrix)

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [29]:
init_weights = [0 for element in list(range(len(all_features)+1))]

In [30]:
weights1e7 = lasso_cyclical_coordinate_descent(normalized_train, train_output,
                                               init_weights, 1e7, 1)

In [31]:
list_feature = [ 'intercept',
                'bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [32]:
for feature, weight in zip(list_feature, weights1e7):
    print('feature: {},  weight: {}'.format(feature,weight))

feature: intercept,  weight: 24429600.609333135
feature: bedrooms,  weight: 0.0
feature: bathrooms,  weight: 0.0
feature: sqft_living,  weight: 48389174.35227978
feature: sqft_lot,  weight: 0.0
feature: floors,  weight: 0.0
feature: waterfront,  weight: 3317511.162719814
feature: view,  weight: 7329961.984896399
feature: condition,  weight: 0.0
feature: grade,  weight: 0.0
feature: sqft_above,  weight: 0.0
feature: sqft_basement,  weight: 0.0
feature: yr_built,  weight: 0.0
feature: yr_renovated,  weight: 0.0


***QUIZ QUESTION***

What features had non-zero weight in this case?

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [33]:
init_weights = [0 for element in list(range(len(all_features)+1))]

weights1e8 = lasso_cyclical_coordinate_descent(normalized_train, train_output,
                                               init_weights, 1e8, 1)

for feature, weight in zip(list_feature, weights1e8):
    print('feature: {},  weight: {}'.format(feature,weight))

feature: intercept,  weight: 71114625.75280938
feature: bedrooms,  weight: 0.0
feature: bathrooms,  weight: 0.0
feature: sqft_living,  weight: 0.0
feature: sqft_lot,  weight: 0.0
feature: floors,  weight: 0.0
feature: waterfront,  weight: 0.0
feature: view,  weight: 0.0
feature: condition,  weight: 0.0
feature: grade,  weight: 0.0
feature: sqft_above,  weight: 0.0
feature: sqft_basement,  weight: 0.0
feature: yr_built,  weight: 0.0
feature: yr_renovated,  weight: 0.0


***QUIZ QUESTION***

What features had non-zero weight in this case?

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [34]:
init_weights = [0 for element in list(range(len(all_features)+1))]

weights1e4 = lasso_cyclical_coordinate_descent(normalized_train, train_output,
                                               init_weights, 1e4, 5e5)

for feature, weight in zip(list_feature, weights1e4):
    print('feature: {},  weight: {}'.format(feature,weight))

feature: intercept,  weight: 77779073.91265233
feature: bedrooms,  weight: -22884012.250233646
feature: bathrooms,  weight: 15348487.080899928
feature: sqft_living,  weight: 92166869.69883084
feature: sqft_lot,  weight: -2139328.082427805
feature: floors,  weight: -8818455.544094978
feature: waterfront,  weight: 6494209.733106552
feature: view,  weight: 7065162.050531973
feature: condition,  weight: 4119079.210067598
feature: grade,  weight: 18436483.52618777
feature: sqft_above,  weight: -14566678.545143452
feature: sqft_basement,  weight: -5528348.751794279
feature: yr_built,  weight: -83591746.20730534
feature: yr_renovated,  weight: 2784276.4601285765


***QUIZ QUESTION***

What features had non-zero weight in this case?

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [35]:
norms = train_norms

normalized_weights1e4 = weights1e4 / norms
normalized_weights1e7 = weights1e7 / norms
normalized_weights1e8 = weights1e8 / norms

normalized_weights1e7[3]

161.3174562483786

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [36]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [37]:
# RSS normalized_weights1e4
errs_1 = (predict_output(test_feature_matrix, normalized_weights1e4) - test_output)**2
RSS_1 = errs_1.sum()
print('RSS: {}'.format(RSS_1))

RSS: 227781004760225.38


In [38]:
# RSS normalized_weights1e7
errs_3 = (predict_output(test_feature_matrix, normalized_weights1e7) - test_output)**2
RSS_3 = errs_3.sum()
print('RSS: {}'.format(RSS_3))

RSS: 275962079909185.28


In [39]:
# RSS normalized_weights1e8
errs_4 = (predict_output(test_feature_matrix, normalized_weights1e8) - test_output)**2
RSS_4 = errs_4.sum()
print('RSS: {}'.format(RSS_4))

RSS: 537166150034084.9


***QUIZ QUESTION***

Which model performed best on the test data?